In [3]:
import json
import random
import numpy as np
import pandas as pd
from datasets import load_from_disk, load_dataset
from datasets import Dataset
from tqdm import tqdm, trange
import kss

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
datasets = load_from_disk("../../data/train_dataset")

In [26]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [27]:
train_datasets = datasets['train']

In [28]:
train_datasets[0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

In [29]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [9]:
def arrow_to_list(data, shuffle=False):
    new = []
    errors = []

    for i in tqdm(range(len(data))):
        split_sentences = kss.split_sentences(data[i]['context'])
        split_sentences_table = {s: i for i, s in enumerate(split_sentences)}
        answer_start = data[i]['answers']['answer_start'][0]

        # 정답이 해당 문장에 들어있는가
        count = 0
        for j in range(len(split_sentences)):
            prev_count = count
            count += len(split_sentences[j])
            if answer_start <= count:
                answer_sentence_index = j  # j번째문장에 정답이 있다.
                break

        # TODO: answer index에 answer이 없는 경우 예외처리
        if data[i]['answers']['text'][0] not in split_sentences[answer_sentence_index]:
            errors.append(i)
        

        if shuffle:
            answer_start_in_sentence = split_sentences[answer_sentence_index].find(data[i]['answers']['text'][0])
            
            random.shuffle(split_sentences)

            shuffle_index = [split_sentences_table[sentence] for sentence in split_sentences]
            shuffle_answer_sentence_index = shuffle_index.index(answer_sentence_index)

            count = 0
            for j in range(shuffle_answer_sentence_index):
                count += len(split_sentences[j]) + 1  # 문장간 띄어쓰기
            shuffle_answer_start = count + answer_start_in_sentence
            shuffle_sentences = ' '.join(split_sentences)

            if shuffle_sentences[shuffle_answer_start] != data[i]['answers']['text'][0][0]:
                errors.append(i)
        else:
            shuffle_sentences = data[i]['context']
            shuffle_answer_start = data[i]['answers']['answer_start'][0]

        new.append({
            'id': data[i]['id'],
            'question': data[i]['question'],
            'context': shuffle_sentences,
            'answers': {
                'text': data[i]['answers']['text'],
                'answer_start': [shuffle_answer_start]
            }
        })
    return new, errors

In [31]:
new_train_datasets, error_datasets = arrow_to_list(train_datasets, shuffle=True)

100%|██████████| 3952/3952 [3:29:25<00:00,  3.18s/it]  


In [33]:
len(error_datasets)

62

In [36]:
a = pd.DataFrame(data=new_train_datasets)
a

,id,question,context,answers
0,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,"그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신...","{'text': ['하원'], 'answer_start': [486]}"
1,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?,특히 1954년 피터 드러커의 저서 《경영의 실제》는 현대적 경영의 기준을 제시하여...,"{'text': ['《경영의 실제》'], 'answer_start': [20]}"
2,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,강희제는 황제로서 자식과 같은 백성들에게 이런 당부의 말을 남겨 황제로서의 도리를 ...,"{'text': ['백성'], 'answer_start': [538]}"
3,mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,"불감은 그 안에 모신 불상의 양식뿐만 아니라, 당시의 건축 양식을 함께 살필 수 있...","{'text': ['중국'], 'answer_start': [504]}"
4,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'text': ['4개'], 'answer_start': [30]}"
...,...,...,...,...
3947,mrc-0-005285,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,"다른 산들은 기울어진 지각 덩어리처럼 보이는데, 이전에 평평했던 땅이 완만하게 기울...","{'text': ['지질 구조'], 'answer_start': [792]}"
3948,mrc-1-000699,애니는 어디서 태어났는가?,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,"{'text': ['뉴저지'], 'answer_start': [494]}"
3949,mrc-0-003429,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,\n \n \n\n1941년에는 악단의 명물이 된 빈 신년음악회가 개최되기 시작했다...,"{'text': ['칼 에케르트'], 'answer_start': [830]}"
3950,mrc-0-003956,드래곤의 암수 구분이 있는 신화는?,\n\n암컷 드래곤은 인간을 미워하여 기후를 자기 맘대로 조종하여 작물을 말라 죽게...,"{'text': ['슬라브 신화'], 'answer_start': [848]}"


In [55]:
Dataset.from_pandas(a).save_to_disk(f"../../sentence_shuffle/train_dataset/train")

In [56]:
with open('../../data/wikipedia_documents.json') as f:
    wiki_data = json.load(f)

In [51]:
print(wiki_data)
print(type(wiki_data))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [57]:
with open('../../sentence_shuffle/wikipedia_documents.json', 'w') as f:
    json.dump(wiki_data, f)

In [6]:
valid_datasets = load_from_disk("../../data/train_dataset/validation")

In [7]:
valid_datasets

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 240
})

In [10]:
new_valid_datasets, error_valid_datasets = arrow_to_list(valid_datasets, shuffle=False)

100%|██████████| 240/240 [12:11<00:00,  3.05s/it]


In [12]:
v_df = pd.DataFrame(data=new_valid_datasets)

In [13]:
v_df

,id,question,context,answers
0,mrc-0-003264,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'text': ['한보철강'], 'answer_start': [284]}"
1,mrc-0-004762,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'text': ['1871년'], 'answer_start': [146]}"
2,mrc-1-001810,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'text': ['나뭇잎'], 'answer_start': [517]}"
3,mrc-1-000219,로타이르가 백조를 구하기 위해 사용한 것은?,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'text': ['금대야'], 'answer_start': [1109]}"
4,mrc-1-000285,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'text': ['수평적 관계'], 'answer_start': [386]}"
...,...,...,...,...
235,mrc-0-000484,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'text': ['제 양왕'], 'answer_start': [1084]}"
236,mrc-0-002095,공놀이 경기장 중 일부는 어디에 위치하고 있나?,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'text': [''일곱 개의 신전 광장''], 'answer_start': [3..."
237,mrc-0-003083,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'text': ['미나미 지로'], 'answer_start': [247]}"
238,mrc-0-002978,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'text': ['200,000명'], 'answer_start': [563]}"


In [15]:
Dataset.from_pandas(v_df).save_to_disk(f"../../sentence_shuffle/train_dataset/validation")

In [65]:
with open('../../data/train_dataset/dataset_dict.json') as f:
    json_data = json.load(f)
with open('../../sentence_shuffle/train_dataset/dataset_dict.json', 'w') as f:
    json.dump(json_data, f)

In [69]:
test_datasets = load_from_disk("../../data/test_dataset/validation")

In [70]:
test_datasets

Dataset({
    features: ['question', 'id'],
    num_rows: 600
})

In [71]:
test_datasets.save_to_disk(f"../../sentence_shuffle/test_dataset/validation")

In [72]:
with open('../../data/test_dataset/dataset_dict.json') as f:
    json_data = json.load(f)
with open('../../sentence_shuffle/test_dataset/dataset_dict.json', 'w') as f:
    json.dump(json_data, f)